In [ ]:
import pandas as pd

file_path = "/content/twcs.csv"
df = pd.read_csv(file_path)

print(df.info())
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59565 entries, 0 to 59564
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   tweet_id                 59565 non-null  int64  
 1   author_id                59565 non-null  object 
 2   inbound                  59565 non-null  bool   
 3   created_at               59565 non-null  object 
 4   text                     59564 non-null  object 
 5   response_tweet_id        40156 non-null  object 
 6   in_response_to_tweet_id  44324 non-null  float64
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 2.8+ MB
None


,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [ ]:
import pandas as pd
import re
file_path = "/content/twcs.csv"
df = pd.read_csv("/content/twcs.csv")

print(df.info())

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


df['cleaned_text'] = df['text'].apply(clean_text)

# Keyword-based categorization
def categorize_query(text):
    if any(word in text for word in ['order', 'shipment', 'tracking']):
        return 'order_status'
    elif any(word in text for word in ['return', 'refund', 'cancel']):
        return 'returns'
    elif any(word in text for word in ['price', 'availability', 'product']):
        return 'product_info'
    elif any(word in text for word in ['error', 'issue', 'support']):
        return 'technical_support'
    else:
        return 'general_inquiry'

# Apply categorization
df['category'] = df['cleaned_text'].apply(categorize_query)

# Display category distribution
print("\nCategory Distribution:")
print(df['category'].value_counts())

# Save categorized data for classification
df[['text', 'cleaned_text', 'category']].to_csv('/mnt/data/categorized_queries.csv', index=False)
print("\nCategorized data saved successfully!")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2811774 entries, 0 to 2811773
Data columns (total 7 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   tweet_id                 int64  
 1   author_id                object 
 2   inbound                  bool   
 3   created_at               object 
 4   text                     object 
 5   response_tweet_id        object 
 6   in_response_to_tweet_id  float64
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 131.4+ MB
None

Category Distribution:
category
general_inquiry      2137468
technical_support     425473
order_status          128821
returns                82888
product_info           37124
Name: count, dtype: int64


OSError: Cannot save file into a non-existent directory: '/mnt/data'

In [ ]:

import pandas as pd

# Load dataset with error handling
file_path = "/content/twcs.csv"

# Use encoding and error handling parameters
try:
    df = pd.read_csv(file_path, encoding='utf-8', on_bad_lines='skip', quoting=3)
    print("Dataset loaded successfully!")
    print(df.info())
except Exception as e:
    print(f"Error loading the dataset: {e}")


<ipython-input-17-bd227c460ce5>:8: DtypeWarning: Columns (0,1,2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='utf-8', on_bad_lines='skip', quoting=3)


Dataset loaded successfully!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3796431 entries, 0 to 3796430
Data columns (total 7 columns):
 #   Column                   Dtype 
---  ------                   ----- 
 0   tweet_id                 object
 1   author_id                object
 2   inbound                  object
 3   created_at               object
 4   text                     object
 5   response_tweet_id        object
 6   in_response_to_tweet_id  object
dtypes: object(7)
memory usage: 202.8+ MB
None


In [ ]:
import re

# Filter only customer queries (inbound=True)
customer_queries = df[df['inbound'] == True].copy()

# Create query categories based on keyword matching
def categorize_query(text):
    text = text.lower()
    if re.search(r'\b(order|purchase|track|status|shipment|delivery)\b', text):
        return 'Order Status'
    elif re.search(r'\b(return|refund|exchange|cancel)\b', text):
        return 'Returns & Refunds'
    elif re.search(r'\b(product|specification|features|availability)\b', text):
        return 'Product Information'
    elif re.search(r'\b(account|login|password|email)\b', text):
        return 'Account Issues'
    elif re.search(r'\b(technical|bug|issue|crash|error)\b', text):
        return 'Technical Support'
    else:
        return 'Other'

# Apply categorization
customer_queries['category'] = customer_queries['text'].apply(categorize_query)

# Clean the text
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'@\w+', '', text)     # Remove mentions
    text = re.sub(r'#\w+', '', text)     # Remove hashtags
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    text = text.lower().strip()
    return text

# Apply text cleaning
customer_queries['clean_text'] = customer_queries['text'].apply(clean_text)

# Display cleaned data
print(customer_queries[['text', 'category', 'clean_text']].head(10))


Empty DataFrame
Columns: [text, category, clean_text]
Index: []


In [ ]:
# Ensure valid data for splitting
if len(customer_queries) > 0:
    X = customer_queries['clean_text']
    y = customer_queries['category']

    # Splitting the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # TF-IDF Vectorization
    vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)

    print("Training Set Size:", X_train_tfidf.shape)
    print("Testing Set Size:", X_test_tfidf.shape)
else:
    print("No data available for training. Check the filtering process.")


No data available for training. Check the filtering process.


In [ ]:
# Check if the DataFrame is empty
print("DataFrame Shape:", customer_queries.shape)
print("Sample rows:")
print(customer_queries.head())

# Count non-empty rows
print("Non-empty rows:", len(customer_queries))


DataFrame Shape: (0, 9)
Sample rows:
Empty DataFrame
Columns: [tweet_id, author_id, inbound, created_at, text, response_tweet_id, in_response_to_tweet_id, category, clean_text]
Index: []
Non-empty rows: 0


In [ ]:
# Ensure there is valid data
if len(customer_queries) > 0:
    X = customer_queries['clean_text']
    y = customer_queries['category']

    # Splitting the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # TF-IDF Vectorization
    vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)

    print("Training Set Size:", X_train_tfidf.shape)
    print("Testing Set Size:", X_test_tfidf.shape)
else:
    print("No data available for training. Check the filtering process.")


No data available for training. Check the filtering process.


In [ ]:
# Display basic info again
print("DataFrame Shape:", customer_queries.shape)

# Check for any missing columns
print("\nColumns in the dataset:")
print(customer_queries.columns)

# Display non-empty rows count
non_empty_rows = customer_queries.dropna(subset=['text']).shape[0]
print(f"\nNon-empty rows (text only): {non_empty_rows}")

# Check for NaN values in 'category' and 'text'
print("\nMissing values:")
print(customer_queries.isna().sum())

# Display a sample of rows, including any NaN categories
print("\nSample rows with NaN categories:")
print(customer_queries[customer_queries['category'].isna()].head(10))


DataFrame Shape: (0, 9)

Columns in the dataset:
Index(['tweet_id', 'author_id', 'inbound', 'created_at', 'text',
       'response_tweet_id', 'in_response_to_tweet_id', 'category',
       'clean_text'],
      dtype='object')

Non-empty rows (text only): 0

Missing values:
tweet_id                   0
author_id                  0
inbound                    0
created_at                 0
text                       0
response_tweet_id          0
in_response_to_tweet_id    0
category                   0
clean_text                 0
dtype: int64

Sample rows with NaN categories:
Empty DataFrame
Columns: [tweet_id, author_id, inbound, created_at, text, response_tweet_id, in_response_to_tweet_id, category, clean_text]
Index: []


In [ ]:
# Read the raw CSV with no filtering, handling bad lines
raw_df = pd.read_csv("/content/twcs.csv", encoding='latin1', on_bad_lines='skip')

# Display the first few rows
print("Raw DataFrame Shape:", raw_df.shape)
print("\nSample Rows:")
print(raw_df.head())

# Check for empty rows and missing values
print("\nEmpty Rows Count:", raw_df.isna().sum())


Raw DataFrame Shape: (2811774, 7)

Sample Rows:
   tweet_id   author_id  inbound                      created_at  \
0         1  sprintcare    False  Tue Oct 31 22:10:47 +0000 2017   
1         2      115712     True  Tue Oct 31 22:11:45 +0000 2017   
2         3      115712     True  Tue Oct 31 22:08:27 +0000 2017   
3         4  sprintcare    False  Tue Oct 31 21:54:49 +0000 2017   
4         5      115712     True  Tue Oct 31 21:49:35 +0000 2017   

                                                text response_tweet_id  \
0  @115712 I understand. I would like to assist y...                 2   
1      @sprintcare and how do you propose we do that               NaN   
2  @sprintcare I have sent several private messag...                 1   
3  @115712 Please send us a Private Message so th...                 3   
4                                 @sprintcare I did.                 4   

   in_response_to_tweet_id  
0                      3.0  
1                      1.0  
2          

In [ ]:
# Drop unnecessary columns with high missing values
df = raw_df.drop(['response_tweet_id', 'in_response_to_tweet_id'], axis=1)

# Filter rows with valid 'text'
df = df[df['text'].notnull() & (df['text'].str.strip() != '')]

# Display dataset shape after cleaning
print("Cleaned DataFrame Shape:", df.shape)

# Create sample category mapping based on keywords (basic classification)
def categorize_text(text):
    text = text.lower()
    if 'bill' in text or 'charge' in text:
        return 'billing'
    elif 'network' in text or 'signal' in text:
        return 'network_issue'
    elif 'refund' in text or 'return' in text:
        return 'refund_request'
    elif 'thank' in text or 'appreciate' in text:
        return 'feedback'
    else:
        return 'other'

# Apply basic categorization
df['category'] = df['text'].apply(categorize_text)

# Display sample rows with categories
print("\nSample Rows with Categories:")
print(df[['text', 'category']].head())

# Check distribution of categories
print("\nCategory Distribution:")
print(df['category'].value_counts())


Cleaned DataFrame Shape: (2811774, 5)

Sample Rows with Categories:
                                                text category
0  @115712 I understand. I would like to assist y...    other
1      @sprintcare and how do you propose we do that    other
2  @sprintcare I have sent several private messag...    other
3  @115712 Please send us a Private Message so th...    other
4                                 @sprintcare I did.    other

Category Distribution:
category
other             2346817
feedback           330048
billing             57058
refund_request      52753
network_issue       25098
Name: count, dtype: int64


model for training :

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Split the dataset into training and testing sets (80/20 split)
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['category'], test_size=0.2, random_state=42)

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))  # Using bigrams and unigrams
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Model Training
model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, y_train)

# Predictions
y_pred = model.predict(X_test_tfidf)

# Model Evaluation
print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))



Accuracy: 0.9963937370522179

Classification Report:
                 precision    recall  f1-score   support

       billing       1.00      0.92      0.96     11288
      feedback       1.00      0.99      0.99     65603
 network_issue       0.99      0.94      0.96      5099
         other       1.00      1.00      1.00    469635
refund_request       0.99      0.97      0.98     10730

      accuracy                           1.00    562355
     macro avg       0.99      0.96      0.98    562355
  weighted avg       1.00      1.00      1.00    562355


Confusion Matrix:
 [[ 10430     88     17    662     91]
 [     1  65135      0    467      0]
 [     0     50   4788    243     18]
 [     1      0     43 469576     15]
 [     0     91      0    241  10398]]


In [ ]:
import joblib

# Save the model and TF-IDF vectorizer
joblib.dump(nb_model, 'naive_bayes_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')
print("Model and vectorizer saved!")


NameError: name 'nb_model' is not defined

In [ ]:
%who


KaggleDatasetAdapter	 LogisticRegression	 TfidfVectorizer	 X	 X_test	 X_test_tfidf	 X_train	 X_train_tfidf	 accuracy_score	 
categorize_query	 categorize_text	 classification_report	 clean_text	 confusion_matrix	 customer_queries	 dataset_url	 df	 file_path	 
joblib	 kagglehub	 model	 non_empty_rows	 np	 pd	 raw_df	 re	 requests	 
train_test_split	 vectorizer	 y	 y_pred	 y_test	 y_train	 


In [ ]:
import joblib

# Save the model and the TF-IDF vectorizer
joblib.dump(model, 'naive_bayes_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')
print("Model and vectorizer saved successfully!")


Model and vectorizer saved successfully!


In [ ]:
import joblib

# Load the model and vectorizer
model = joblib.load('naive_bayes_model.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')

# Sample prediction
sample_query = ["I want a refund for my recent order."]
sample_tfidf = vectorizer.transform(sample_query)
category = model.predict(sample_tfidf)
print("Predicted Category:", category[0])


Predicted Category: refund_request


In [ ]:
!pip install Flask joblib


In [ ]:
from flask import Flask, request, jsonify
import joblib

app = Flask(__name__)

# Load the saved model and vectorizer
model = joblib.load("model/naive_bayes_model.pkl")
vectorizer = joblib.load("model/tfidf_vectorizer.pkl")

@app.route('/')
def home():
    return "Customer Query Classification API"

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()
        query = data.get('query')

        if not query:
            return jsonify({"error": "No query provided"}), 400

        # Vectorize the input query
        query_tfidf = vectorizer.transform([query])

        # Make the prediction
        prediction = model.predict(query_tfidf)[0]

        # Return the predicted category
        return jsonify({"query": query, "category": prediction})

    except Exception as e:
        return jsonify({"error": str(e)}), 500


if __name__ == '__main__':
    app.run(debug=True)


FileNotFoundError: [Errno 2] No such file or directory: 'model/naive_bayes_model.pkl'

In [ ]:
from flask import Flask, request, jsonify
import joblib

app = Flask(__name__)

# Load the saved model and vectorizer
model = joblib.load("model/naive_bayes_model.pkl")
vectorizer = joblib.load("model/tfidf_vectorizer.pkl")

@app.route('/')
def home():
    return "Customer Query Classification API"

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()
        query = data.get('query')

        if not query:
            return jsonify({"error": "No query provided"}), 400

        # Vectorize the input query
        query_tfidf = vectorizer.transform([query])

        # Make the prediction
        prediction = model.predict(query_tfidf)[0]

        # Return the predicted category
        return jsonify({"query": query, "category": prediction})

    except Exception as e:
        return jsonify({"error": str(e)}), 500


if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
from flask import Flask, request, jsonify
import joblib
from pyngrok import ngrok

app = Flask(__name__)

# Load the saved model and vectorizer
model = joblib.load("/content/naive_bayes_model.pkl")
vectorizer = joblib.load("/content/tfidf_vectorizer.pkl")

# Start ngrok tunnel
port = 5000
public_url = ngrok.connect(port).public_url
print(f"🔗 Public URL: {public_url}")

@app.route('/')
def home():
    return "Customer Query Classification API"

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()
        query = data.get('query')

        if not query:
            return jsonify({"error": "No query provided"}), 400

        # Vectorize the input query
        query_tfidf = vectorizer.transform([query])

        # Make the prediction
        prediction = model.predict(query_tfidf)[0]

        # Return the predicted category
        return jsonify({"query": query, "category": prediction})

    except Exception as e:
        return jsonify({"error": str(e)}), 500


if __name__ == '__main__':
    app.run(port=port)


In [ ]:
!pip install pyngrok
